## Set up

In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy 
import pandas
from scipy import stats
import time
from itertools import combinations

In [4]:
url = "https://tfhub.dev/google/universal-sentence-encoder/4"
nlp = hub.load(url)
print ("module %s loaded" % url)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [5]:
nlp(['communication skills'])

<tf.Tensor: shape=(1, 512), dtype=float32, numpy=
array([[ 0.00847739, -0.02442496, -0.00858844, -0.01480304, -0.00306875,
         0.05746894,  0.05213621,  0.01705379,  0.03088988,  0.02509503,
        -0.01306393,  0.06517093,  0.02921114, -0.05873831,  0.01662105,
        -0.08658601,  0.07640935,  0.01744252,  0.07326126,  0.05524365,
        -0.02530596, -0.0477285 , -0.01923554, -0.04782432, -0.08482939,
        -0.03659053,  0.0323831 , -0.06681018, -0.06364907,  0.00755073,
         0.03215759, -0.01605681, -0.08077275, -0.03591077, -0.04987152,
        -0.00708391,  0.00600367,  0.03089833, -0.0008762 ,  0.03288973,
        -0.0340079 ,  0.05709708,  0.0520334 , -0.06192387,  0.01787907,
        -0.04159049, -0.00130487, -0.0392967 , -0.00623377, -0.05834588,
        -0.02470777,  0.0453537 ,  0.03174945, -0.03971299,  0.03406378,
        -0.00534536, -0.01546777, -0.04535134,  0.08519338,  0.05317898,
        -0.00199912,  0.05836201, -0.00169684, -0.03763165, -0.02187691,
 

In [7]:
skills = ['additive manufacturing','problem solving','communication skills']
skills_e = nlp(skills)
print(skills_e)
numpy.inner(skills_e,skills_e)

tf.Tensor(
[[ 0.04736473 -0.00548766  0.01081967 ... -0.06126872 -0.0060198
  -0.00647844]
 [-0.02010326 -0.07038264  0.01779231 ... -0.05918299 -0.00423545
  -0.02273482]
 [ 0.00847739 -0.02442496 -0.00858843 ... -0.04444852 -0.06791332
   0.03621201]], shape=(3, 512), dtype=float32)


array([[0.99999964, 0.22803646, 0.25098065],
       [0.22803646, 1.0000005 , 0.33013391],
       [0.25098065, 0.33013391, 0.99999976]], dtype=float32)

In [4]:
print(numpy.inner(skills_e,skills_e)[0][1], 'similarity between additive manufacturing and problem solving')
print(numpy.inner(skills_e,skills_e)[0][2], 'similarity between additive manufacturing and communication skills')
print(numpy.inner(skills_e,skills_e)[1][2], 'similarity between communication skills and problem solving')

0.22803646 similarity between additive manufacturing and problem solving
0.25098065 similarity between additive manufacturing and communication skills
0.33013391 similarity between communication skills and problem solving


## Data

In [8]:
filepath = 'D:/BG/Data/Processing/4_data.txt'
data = pandas.read_csv(filepath, sep='\t')
print(data.shape[0],'postings in full sample')
ourSB = pandas.read_csv('SB.csv', sep=',')
ourSB['plant'] = ourSB.firm + ' ' + ourSB.City
a = ourSB.plant.unique().tolist()
print(len(a),'identified SB plants')
b = data.loc[(data.plant_5==0)&(data.plant.isin(a)),'plant'].unique().tolist()
print(len(b),'hybrid SB plants')
print(b)
c = data.loc[(data.plant_5.notnull())&(data.plant.isin(b)),'firm'].unique().tolist()
print(len(c),'SB-B firms')
d = data.loc[(data.plant_5.notnull())&(data.firm.isin(c))]
print(d.groupby(['TECH','firm']).nunique()[['BGTJobId']])
c = [x for x in c if x not in ['Arconic Foundati','Carpenter Technolo','Jabil Circuit']]
SBB = data[(data.plant_5.notnull())&(data.firm.isin(c))]
print(SBB.shape[0],'postings in sample SB-B')
print(SBB.loc[SBB.plant_5==2,'plant'].nunique(),'pure TM plants in sample SB-B')
print(SBB.loc[SBB.plant_5==1,'plant'].nunique(),'pure AM plants in sample SB-B')
print(SBB.loc[SBB.plant_5==0,'plant'].nunique(),'hybrid plants in sample SB-B')
print(SBB.firm.nunique(),'SB-B firms')
print(SBB.plant.nunique(),'SB-B plants')
print(SBB.plant_5.value_counts())
print(SBB.TECH.value_counts())
N_jobs = SBB.groupby(['occupation','TECH']).nunique()[['BGTJobId']].reindex(['Manager','Engineer','Technician','Operator'], level='occupation').unstack(level=[1])
N_jobs.columns = N_jobs.columns.droplevel(0)
print(N_jobs)

3091473 postings in full sample
85 identified SB plants
25 hybrid SB plants
['Argen San Diego', 'Align Technolo San Jose', 'Protolabs Maple Plain', 'Stratasys Direct Eden Prairie', 'Jabil Circuit San Jose', 'Jabil Circuit Saint Petersburg', 'Jabil Circuit Anaheim', 'Protolabs Raleigh', 'Carpenter Technolo Bridgeville', 'Oerlikon Group Westbury', 'Stratasys Direct Minneapolis', 'Stratasys Direct New York', 'Stratasys Direct Poway', 'Stratasys Direct Santa Clarita', 'Shapeways New York', 'Carbon 3d Redwood City', 'Arconic Foundati New Kensington', 'Arconic Foundati Austin', 'Protolabs Morrisville', 'Stratasys Direct Austin', 'Stratasys Direct Belton', 'Xometry Gaithersburg', 'Fast Radius Chicago', 'Exone Irwin', 'Jabil Circuit Albuquerque']
13 SB-B firms
                         BGTJobId
TECH firm                        
AM   Align Technolo             9
     Arconic Foundati          48
     Argen                     11
     Carbon 3d                 85
     Carpenter Technolo        47

## Example 1 - An engineer in Protolabs

In [41]:
df = SBB.copy()
df['Year'] = pandas.to_datetime(df.JobDate).dt.year
A = df.loc[(df.firm=='Protolabs')&(df.occupation=='Engineer')&(df.Year==2019),'SK']
A = A.iloc[0]
A = A.split(',')
print('An engineer in Protolabs:',A)
pairs = list(set(combinations(A,2)))
print('All possible pairs:',pairs)
for pair in pairs:
  token1 = nlp([pair[0]])
  token2 = nlp([pair[1]])
  score = numpy.inner(token1,token2)
  scores.append(score)
avg_scores = sum(scores)/len(scores)
print('Similarity score for an engineer in Protoloabs:',avg_scores[0][0])

An engineer in Protolabs: ['microsoft office', 'written communication', 'scheduling', 'onboarding', 'process improvement', 'quality assurance and control', 'engineering management', 'height gauges', 'quality management', 'problem solving', 'training materials', 'international traffic in arms regulations (itar)', 'calipers', 'computer numerical control (cnc)']
All possible pairs: [('microsoft office', 'computer numerical control (cnc)'), ('problem solving', 'computer numerical control (cnc)'), ('onboarding', 'computer numerical control (cnc)'), ('engineering management', 'computer numerical control (cnc)'), ('microsoft office', 'problem solving'), ('onboarding', 'international traffic in arms regulations (itar)'), ('engineering management', 'international traffic in arms regulations (itar)'), ('scheduling', 'calipers'), ('microsoft office', 'height gauges'), ('microsoft office', 'onboarding'), ('process improvement', 'calipers'), ('written communication', 'calipers'), ('process improvem

## Example 2 - An operator in Protolabs

In [43]:
df = SBB.copy()
df['Year'] = pandas.to_datetime(df.JobDate).dt.year
A = df.loc[(df.firm=='Protolabs')&(df.occupation=='Operator')&(df.Year==2019),'SK']
A = A.iloc[0]
A = A.split(',')
print('An operator in Protolabs:',A)
pairs = list(set(combinations(A,2)))
print('All possible pairs:',pairs)
for pair in pairs:
  token1 = nlp([pair[0]])
  token2 = nlp([pair[1]])
  score = numpy.inner(token1,token2)
  scores.append(score)
avg_scores = sum(scores)/len(scores)
print('Similarity score for an operator in Protoloabs:',avg_scores[0][0])

An operator in Protolabs: ['welding equipment', 'international traffic in arms regulations (itar)', 'organizational skills', '3d modeling / design', 'welding', 'communication skills', 'mig and tig welding']
All possible pairs: [('international traffic in arms regulations (itar)', 'welding'), ('welding equipment', 'welding'), ('welding', 'communication skills'), ('international traffic in arms regulations (itar)', 'communication skills'), ('welding equipment', 'communication skills'), ('3d modeling / design', 'mig and tig welding'), ('organizational skills', '3d modeling / design'), ('international traffic in arms regulations (itar)', '3d modeling / design'), ('welding equipment', '3d modeling / design'), ('communication skills', 'mig and tig welding'), ('organizational skills', 'mig and tig welding'), ('welding equipment', 'organizational skills'), ('international traffic in arms regulations (itar)', 'organizational skills'), ('welding', 'mig and tig welding'), ('international traffic 

## Similarity

In [44]:
mydata = SBB.copy()
mydata['SK'] = mydata.SK.str.lower()
mydata['SK'] = mydata.SK.str.split(',')
avg_scores = []
start = time.time()
for i,row in zip(mydata['BGTJobId'],mydata['SK']):
  scores = []
  pairs = list(set(combinations(row,2)))
  for pair in pairs:
    token1 = nlp([pair[0]])
    token2 = nlp([pair[1]])
    score = numpy.inner(token1,token2)
    scores.append(score)
  avg_scores.append([i,float(sum(scores)/len(scores))])
end = time.time()
print(f'Finished in {end-start} seconds')

Finished in 1044.442454814911 seconds


In [8]:
df = pandas.DataFrame(avg_scores,columns=['BGTJobId','Similarity'])
DF = pandas.merge(left=SBB,right=df,how='inner',on='BGTJobId')
print(DF.columns)

Index(['BGTJobId', 'JobDate', 'CleanTitle', 'CanonTitle', 'SOC', 'SOCName',
       'ONET', 'ONETName', 'Employer', 'Sector', 'SectorName', 'NAICS3',
       'NAICS4', 'NAICS5', 'NAICS6', 'City', 'County', 'State', 'Lat', 'Lon',
       'Edu', 'MaxEdu', 'Degree', 'MaxDegree', 'Exp', 'MaxExp', 'MinSalary',
       'MaxSalary', 'MinHrlySalary', 'MaxHrlySalary', 'PayFrequency',
       'SalaryType', 'JobHours', 'SOC2', 'SOC4', 'SOC5', 'SK',
       'Employer_clean', 'n_terms', 'SK_without', 'TECH', 'occupation',
       'firm1', 'firm2', 'firm', 'plant', 'plant_all', 'plant_5',
       'development', 'materials', 'design', 'inventory', 'tooling',
       'automation', 'production', 'maintenance', 'technical',
       'administrative', 'management', 'finance', 'business', 'data',
       'software', 'office', 'ml', 'cognitive', 'creativity', 'social',
       'character', 'customer', 'writing', 'nonroutine analytic',
       'nonroutine manual', 'routine cognitive', 'routine manual',
       'sequential

## Firm means

In [40]:
occupation = ['Manager','Engineer','Technician','Operator']
df = DF[['firm','TECH','occupation','Similarity']]

mymean = df.groupby(by=['firm','occupation','TECH']).mean().reset_index()
t = []
for o in occupation:
  t.append([o,stats.ttest_ind(mymean.loc[(mymean.TECH=='AM')&(mymean.occupation==o),'Similarity'],mymean.loc[(mymean.TECH=='TM')&(mymean.occupation==o),'Similarity'],equal_var=False)[1]])
tdf = pandas.DataFrame(t,columns=['occupation','p-val'])
tdf.set_index(['occupation'],inplace=True)
 
mean_mymean = mymean.groupby(by=['occupation','TECH']).mean().unstack(level=0).T
mean_mymean = mean_mymean.droplevel(0,axis=0)
table2 = pandas.merge(left=mean_mymean,right=tdf,how='inner',left_on=['occupation'],right_on=['occupation'])
table2['statistics'] = 'Mean'
table2.set_index('statistics',append=True,inplace=True)

mysd = df.groupby(by=['firm','occupation','TECH']).std(ddof=0).reset_index()
mean_mysd = mysd.groupby(by=['occupation','TECH']).mean().unstack(level=0).T
mean_mysd = mean_mysd.droplevel(0,axis=0)
mean_mysd['statistics'] = 'SD'
mean_mysd.set_index('statistics',append=True,inplace=True)
mean_mysd['p-val'] = numpy.nan
table2 = pandas.concat([table2,mean_mysd],axis=0).sort_index(kind='merge').unstack('occupation').swaplevel(0,1,axis=1).reindex(occupation,axis=1,level=0)
table2.rename_axis([None,None],axis=1,inplace=True)

postings = SBB.groupby(['occupation','TECH']).nunique()[['BGTJobId']].reindex(occupation, level='occupation').T
postings.rename_axis([None,None],axis=1,inplace=True)
postings.rename(index={'BGTJobId':'Number of job postings'},inplace=True)
plants = SBB.groupby(['occupation','TECH']).nunique()[['plant']].reindex(occupation, level='occupation').T
plants.rename_axis([None,None],axis=1,inplace=True)
plants.rename(index={'plant':'Number of plants'},inplace=True)
table2 = pandas.concat([table2,postings,plants],axis=0).reindex(occupation,axis=1,level=0)

table = table2.copy()

mylist = [('Manager','AM'),('Manager','TM'),('Engineer','AM'),('Engineer','TM'),('Technician','AM'),('Technician','TM'),('Operator','AM'),('Operator','TM')]
table[mylist] = table[mylist].applymap('{:.3f}'.format)

for i in ['Manager','Engineer','Technician','Operator']:
  table.loc[(table[(i,'p-val')]<0.1)&(table[(i,'p-val')]>=0.05),(i,'AM')] = table.loc[(table[(i,'p-val')]<0.1)&(table[(i,'p-val')]>=0.05),(i,'AM')]+'*'
  table.loc[(table[(i,'p-val')]<0.05)&(table[(i,'p-val')]>=0.01),(i,'AM')] = table.loc[(table[(i,'p-val')]<0.05)&(table[(i,'p-val')]>=0.01),(i,'AM')]+'**'
  table.loc[table[(i,'p-val')]<0.01,(i,'AM')] = table.loc[table[(i,'p-val')]<0.01,(i,'AM')]+'***'

table = table[mylist]
table.loc['SD'] = table.loc['SD'].apply(lambda x: '(' + x + ')')
table = table.mask(table == '(nan)',numpy.nan)
table = table.mask(table == 'nan',numpy.nan)
table = table.mask(table == '-0.000','0.000')
table.loc['Number of job postings'] = table.loc['Number of job postings'].replace(r'.000','',regex=True)
table.loc['Number of plants'] = table.loc['Number of plants'].replace(r'.000','',regex=True)

table


Manager          Engineer          Technician  \
                              AM       TM       AM       TM         AM   
Mean                    0.208***    0.244    0.199    0.215      0.149   
SD                       (0.033)  (0.036)  (0.018)  (0.024)    (0.016)   
Number of job postings       172      397      144      131         43   
Number of plants              17       34       15       28         12   

                                Operator           
                             TM       AM       TM  
Mean                      0.198  0.179**    0.214  
SD                      (0.029)  (0.021)  (0.032)  
Number of job postings       88       79      387  
Number of plants             20       15       35